**Initial Configuration**

In [2]:
import sys
sys.path.append('..')

In [3]:
import nppes

**Begin Notebook Code**

# Get data from API

In [10]:
import requests
from pandas.io.json import json_normalize

In [6]:
API_columns = [
    'number',
    'enumeration_type',
    'taxonomy_description',
    'first_name',
    'last_name',
    'organization_name',
    'address_purpose',
    'city',
    'state',
    'postal_code',
    'country_code',
    'limit',
    'skip',
    'pretty'
    ]

In [3]:
payload = {'version': '2.1', 'last_name': 'Gomez', 'first_name': 'Julia'}
r = requests.get('https://npiregistry.cms.hhs.gov/api/?', params=payload)

In [8]:
json_data = r.json()

In [17]:
type(json_data)

dict

In [16]:
json_normalize(json_data['results'][0]['addresses'])

<ipython-input-16-9c09b314637d>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  json_normalize(json_data['results'][0]['addresses'])


,country_code,country_name,address_purpose,address_type,address_1,address_2,city,state,postal_code,telephone_number,fax_number
0,US,United States,LOCATION,DOM,2400 PATTERSON ST,500,NASHVILLE,TN,372031562,615-327-7400,615-327-4818
1,US,United States,MAILING,DOM,2400 PATTERSON ST,500,NASHVILLE,TN,372031562,615-327-7400,615-327-4818
